This notebook prepares the Fingerprint training data for the Kernel Metric Network (KMN).

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from tqdm import tqdm
from rdkit.Chem import AllChem,DataStructs
import matplotlib.pyplot as plt
import pickle
from typing import List,Tuple
from rdkit import RDLogger     
from faiss import write_index, read_index
RDLogger.DisableLog('rdApp.*')   
import pickle
import pandas as pd


# Loading the Pistachio database
df = pickle.load(open('FPCompatible_Cleaned_Pistachio.pkl','rb'))
df = df[~df['namerxndef'].isna()].reset_index(drop=True) # dropping any reaction that does not have a classification





In [ ]:
# Here is my first filter. The three conditions are 1. cannot be a reaction that does nothing, 2. products cannot be a subset of the reactants and 3. the reactants cannot be a subset of the products

reactions = df['reaction'].values
clean_reactions = []
for i in tqdm(range(len(reactions))):
    split_result = reactions[i].split('>')
    left = split_result[0]
    right =  split_result[-1]
    
    
    left_set = set(left.split('.'))
    right_set = set(right.split('.'))
    condition1 = not(left_set <= right_set) # products are not the subset of the reactants
    condition2 = not(right_set <= left_set) # ractants are not the subset of the products
    condition3 = condition1 and condition2
    
    
    if left != right and condition3: # If this reaction is not "doing nothing" and fullfill condition3
        
        cleaned = left + '>>' + right
        
        clean_reactions.append(cleaned)
        
print('Non-Unique Rxns',len(clean_reactions))
clean_reactions_set = set(clean_reactions)
#clean_reactions = list(clean_reactions)
print('Unique Rxns', len(clean_reactions_set))
del(clean_reactions_set)

# I recorded the indices of the passed reactions and created the "Clean_Pistachio.csv" which is then used to generate the final FP-compatible dataframe.


# You can use this code to filter down your database and save it as "FPCompatible_Cleaned_Pistachio.pkl".

In [ ]:
# Defining the function that calculates reaction fingerprints
from rdkit.Chem import RDKFingerprint, AllChem


def create_rxn_MixFP(rxn, rxnfpsize=16384, pfpsize=16384, useFeatures=False, calculate_rfp=True, useChirality=False):

    
    fpgen = AllChem.GetRDKitFPGenerator(maxPath=4,fpSize=rxnfpsize)
    
    rsmi = rxn.split('>>')[0]
    psmi = rxn.split('>>')[1]
    rct_mol = Chem.MolFromSmiles(rsmi)
    prd_mol = Chem.MolFromSmiles(psmi)
    
    rsmi = rsmi.encode('utf-8')
    psmi = psmi.encode('utf-8')
    try:
        mol = Chem.MolFromSmiles(rsmi)
    except Exception as e:
        print(e)
        return
    try:
        fp_bit = AllChem.GetMorganFingerprintAsBitVect(
            mol=mol, radius=2, nBits=rxnfpsize, useFeatures=useFeatures, useChirality=useChirality)
        fp = np.empty(rxnfpsize, dtype=int)
        DataStructs.ConvertToNumpyArray(fp_bit, fp)
        
        rdkitfp = np.array(fpgen.GetFingerprint(mol).ToList())
        fp = np.concatenate((rdkitfp,fp), axis = -1) # RDKIT(p=4)+ Morgan(r=2) fingerprint
        
    except Exception as e:
        #print("Cannot build reactant fp due to {}".format(e))
        return
    rfp = fp
    
    try:
        mol = Chem.MolFromSmiles(psmi)
    except Exception as e:
        return
    try:
        fp_bit = AllChem.GetMorganFingerprintAsBitVect(
            mol=mol, radius=2, nBits=pfpsize, useFeatures=useFeatures, useChirality=useChirality)
        fp = np.empty(pfpsize, dtype=int)
        DataStructs.ConvertToNumpyArray(fp_bit, fp)
        rdkitfp = np.array(fpgen.GetFingerprint(mol).ToList())
        
        fp = np.concatenate((rdkitfp,fp), axis = -1)
        
    except Exception as e:
        #print("Cannot build product fp due to {}".format(e))
        return
    pfp = fp
    return [pfp, rfp, pfp-rfp]    


def create_rxn_MorganFP(rxn, rxnfpsize=16384, pfpsize=16384, useFeatures=False, calculate_rfp=True, useChirality=False):

    rsmi = rxn.split('>>')[0]
    psmi = rxn.split('>>')[1]
    rct_mol = Chem.MolFromSmiles(rsmi)
    prd_mol = Chem.MolFromSmiles(psmi)
    
    rsmi = rsmi.encode('utf-8')
    psmi = psmi.encode('utf-8')
    try:
        mol = Chem.MolFromSmiles(rsmi)
    except Exception as e:
        print(e)
        return
    try:
        fp_bit = AllChem.GetMorganFingerprintAsBitVect(
            mol=mol, radius=2, nBits=rxnfpsize, useFeatures=useFeatures, useChirality=useChirality)
        fp = np.empty(rxnfpsize, dtype='float32')
        DataStructs.ConvertToNumpyArray(fp_bit, fp)
    except Exception as e:
        #print("Cannot build reactant fp due to {}".format(e))
        return
    rfp = fp
    
    try:
        mol = Chem.MolFromSmiles(psmi)
    except Exception as e:
        return
    try:
        fp_bit = AllChem.GetMorganFingerprintAsBitVect(
            mol=mol, radius=2, nBits=pfpsize, useFeatures=useFeatures, useChirality=useChirality)
        fp = np.empty(pfpsize, dtype='float32')
        DataStructs.ConvertToNumpyArray(fp_bit, fp)
    except Exception as e:
        #print("Cannot build product fp due to {}".format(e))
        return
    pfp = fp
    return [pfp, rfp, pfp-rfp]    


def create_rxn_RdkitFP(rxn, rxnfpsize=16384, pfpsize=16384, useFeatures=False, calculate_rfp=True, useChirality=False):
    
    fpgen = AllChem.GetRDKitFPGenerator(maxPath=4,fpSize=rxnfpsize)

    rsmi = rxn.split('>>')[0]
    psmi = rxn.split('>>')[1]
    rct_mol = Chem.MolFromSmiles(rsmi)
    prd_mol = Chem.MolFromSmiles(psmi)
    
    rsmi = rsmi.encode('utf-8')
    psmi = psmi.encode('utf-8')
    try:
        mol = Chem.MolFromSmiles(rsmi)
    except Exception as e:
        print(e)
        return
    try:
        fp = np.array(fpgen.GetFingerprint(mol).ToList())
    except Exception as e:
        #print("Cannot build reactant fp due to {}".format(e))
        return
    rfp = fp
    
    try:
        mol = Chem.MolFromSmiles(psmi)
    except Exception as e:
        return
    try:
        fp = np.array(fpgen.GetFingerprint(mol).ToList())
    except Exception as e:
        #print("Cannot build product fp due to {}".format(e))
        return
    pfp = fp
    return [pfp, rfp, pfp-rfp]    



In [ ]:
fps = create_rxn_MixFP('C1CNCCC12CCCCC2.C1CNCCC12CCCCC2.C1CNCCC12CCCCC2>>C1CNCCC12CCCCC2.C3CNCCC34CCCCC4.C5CNCCC56CCCCC6',
                        rxnfpsize = 1024,
                        pfpsize = 1024,
                        useChirality=True )

fps = create_rxn_MorganFP('C1CNCCC12CCCCC2.C1CNCCC12CCCCC2.C1CNCCC12CCCCC2>>C1CNCCC12CCCCC2.C3CNCCC34CCCCC4.C5CNCCC56CCCCC6',
                        rxnfpsize = 1024,
                        pfpsize = 1024,
                        useChirality=True )

fps = create_rxn_RdkitFP('C1CNCCC12CCCCC2.C1CNCCC12CCCCC2.C1CNCCC12CCCCC2>>C1CNCCC12CCCCC2.C3CNCCC34CCCCC4.C5CNCCC56CCCCC6',
                        rxnfpsize = 1024,
                        pfpsize = 1024,
                        useChirality=True )


print('Executed.')

In [ ]:
len(clean_reactions)

In [ ]:
import numpy as np
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from functools import partial

# The following is executed on a single CPU node with 128 cores/256 threads and 512 GB RAM. 

def process_single_reaction(rxn, fp_function, fp_size=1024, useChirality=True):
    """
    Process a single reaction and create its fingerprint.
    Returns the fingerprint features or None if failed.
    """
    try:
        rxn_fp = fp_function(rxn, rxnfpsize=fp_size, pfpsize=fp_size, useChirality=useChirality)
        
        if rxn_fp is not None:
            # Concatenate the fingerprints
            features = np.concatenate((rxn_fp[1], rxn_fp[2], rxn_fp[0]), axis=-1)
            return features
        return None
    except Exception as e:
        print(f'Failed with error: {str(e)}')
        return None

def process_reaction_batch(batch_data):
    """
    Process a batch of reactions.
    batch_data: tuple of (batch_idx, reactions)
    Returns: tuple of (batch_idx, list of (index, feature))
    """
    batch_idx, reactions = batch_data
    results = []
    
    for i, rxn in enumerate(reactions):
        absolute_idx = batch_idx + i
        result = process_single_reaction(rxn, fp_function = create_rxn_MixFP) # You need to change this funciton to create different fingerprint encodings 
        
        results.append((absolute_idx, result))  # Keep all results, including None
            
    return batch_idx, results

def parallel_fingerprint_generation(clean_reactions, batch_size=100, n_processes=None):
    """
    Generate fingerprints in parallel using multiprocessing.
    Maintains the exact ordering of input reactions in the output.
    
    Parameters:
    -----------
    clean_reactions : list
        List of reactions to process
    batch_size : int
        Size of batches to process in parallel
    n_processes : int, optional
        Number of processes to use. Defaults to CPU count - 1
        
    Returns:
    --------
    features : np.ndarray
        Array of concatenated fingerprints, in the same order as input reactions
        (only successful ones)
    successful_indices : list
        List of indices of successfully processed reactions, in order
    """
    if n_processes is None:
        n_processes = max(1, cpu_count() - 1)
    
    # Create batches of reactions
    batches = []
    for i in range(0, len(clean_reactions), batch_size):
        batch_reactions = clean_reactions[i:i + batch_size]
        batches.append((i, batch_reactions))
    
    # Process batches in parallel
    all_results = []
    
    with Pool(processes=n_processes) as pool:
        # Use tqdm to show progress
        for batch_idx, batch_results in tqdm(pool.imap(process_reaction_batch, batches), 
                                           total=len(batches)):
            all_results.append((batch_idx, batch_results))
    
    # Sort results by batch index to maintain order
    all_results.sort(key=lambda x: x[0])
    
    # Flatten and filter results while maintaining order
    features = []
    successful_indices = []
    
    for batch_idx, batch_results in all_results:
        for idx, feature in batch_results:
            if feature is not None:
                features.append(feature)
                successful_indices.append(idx)
    
    # Convert to numpy array
    features = np.array(features)
    
    return features, successful_indices

# Set your parameters
batch_size = 100  # Adjust based on your memory constraints
n_processes = None  # Will use CPU count - 1

# Run the parallel processing
features, successful_indices = parallel_fingerprint_generation(
    clean_reactions,
    batch_size=batch_size,
    n_processes=n_processes
)

# Print results
print(f"Successfully processed {len(successful_indices)} reactions")
print(f"Feature dimension: {features.shape[1]}")

# Verify ordering
print("Successful indices are in order:", 
      all(successful_indices[i] <= successful_indices[i+1] 
          for i in range(len(successful_indices)-1)))

In [ ]:
features.shape

In [ ]:
features = features.astype(np.int8) # To make it IO-efficient
pickle.dump(features, open('MixFP_Features_p4_r2_1024_dim.pkl', 'wb'))